In [2]:
import networkx as nx
from graph_tool.all import *


In [3]:
from typing import Set
import graph_tool.all as gt

def get_graphtools_graph(g:nx.Graph, only_from_nodes:Set = set()):
    g = nx.convert_node_labels_to_integers(g)
    graph = gt.Graph(directed=False)
    graph.set_fast_edge_removal(True)
    graph.vertex_properties["node"] = graph.new_vertex_property("object")
    # for n in self.nodes_set:
    #     graph.add_edge_list([(n,n2) for n2 in self.get_connections(n)])
    if len(only_from_nodes)!=0:
        processed = set(self.nodes).difference(only_from_nodes)
    else:
        processed = set()
    edge_list = []
    
    for n in g.nodes():
        for c in g.neighbors(n):
            if c not in processed:
                edge_list.append((int(n), int(c)))
                graph.vertex_properties["node"][c] = c
            graph.vertex_properties["node"][n] = n
        processed.add(n)
    graph.add_edge_list(edge_list)
    # for n in self.nodes:
    #     for c in self.get_connections(n):
    #         if c not in processed:
    #             graph.add_edge(n, c)
    #             graph.vertex_properties["node"][c] = c
    #         graph.vertex_properties["node"][n] = n

    return graph

In [4]:
# from copy import deepcopy
def get_core_strengths(graph:nx.Graph, core_numbers):
    cs = dict()
    for node in graph:
        bn = [ x for x in graph.neighbors(node) if core_numbers[x]>=core_numbers[node]]
        cs[node] = len(bn) - core_numbers[node]+1
    return cs

In [5]:
class GraphTransformation:
    def __init__(self, edges2add, edges2remove, transform_type = 'add', legal=True):
        self.edges_add = frozenset([frozenset(e) for e in edges2add])
        self.edges_remove = frozenset([frozenset(e) for e in edges2remove])
        self.type = transform_type
        self.legal = legal
    
    def __eq__(self, other):
        return self.__class__ == other.__class__ \
        and self.edges_add == other.edges_add\
        and self.edges_remove == other.edges_remove
    
    def __hash__(self):
        return hash(self.edges_add)+hash(self.edges_remove)
    
    def __str__(self):
        return "+{}, -{}".format(str(self.edges_add), str(self.edges_remove))
    
    def __repr__(self):
        return self.__str__()
    
    def __len__(self):
        return len(self.edges_add)+len(self.edges_remove)

In [6]:
# hash(frozenset([5,10]))

In [7]:
# hash(frozenset([10,5]))

In [8]:
from itertools import product
def recompute_cs(edges_add, edges_remove, k, cs_start):
    cs = cs_start.copy()
    for e1_cs in edges_add:
        e = list(e1_cs)
        if k[e[0]]<k[e[1]]:
            cs[e[0]]+=1
        if k[e[0]]>k[e[1]]:
            cs[e[1]]+=1
        if k[e[0]]==k[e[1]]:
            cs[e[0]]+=1
            cs[e[1]]+=1
    for e1_cs in edges_remove:
        e = list(e1_cs)
        if k[e[0]]<k[e[1]]:
            cs[e[0]]-=1
        if k[e[0]]>k[e[1]]:
            cs[e[1]]-=1
        if k[e[0]]==k[e[1]]:
            cs[e[0]]-=1
            cs[e[1]]-=1
    return cs 

def transforms_join(t1, t2):
    assert(t1.type==t2.type)
    edges_add = set(t1.edges_add).union(t2.edges_add)
    edges_remove = set(t1.edges_remove).union(t2.edges_remove)
    return GraphTransformation(edges_add, edges_remove, t1.type)

def transformations_product(l1,l2):
    transformations_set = set()
    for t1 in l1:
        for t2 in l2:
            transformations_set.add(transforms_join(t1, t2))
    return transformations_set



In [9]:
# a = GraphTransformation([(1,2),(2,5),(3,4)],[(1,6)])
# b = GraphTransformation([(1,2),(2,5),(3,4)],[(1,60)])
# c = GraphTransformation([(2,6)],[(1,6)])
# d = GraphTransformation([(2,69)],[(1,0)])
# f = GraphTransformation([(2,69)],[(1,0),(1,3),(0,2)])
# pdct = transformations_product([a,b], [c,d, f])
# print(len(pdct))

In [10]:

def test_minimal(graph:nx.Graph, core_numbers):
    if (nx.core_number(graph)!=core_numbers):
        return False
    cs = get_core_strengths(graph, core_numbers)
    edge_candidates = [(u,v) for u,v in graph.edges() if 
                           (core_numbers[u]==core_numbers[v] and cs[u]>1 and cs[v]>1)
                           or (core_numbers[u]>core_numbers[v] and cs[v]>1) or (core_numbers[u]<core_numbers[v] and cs[u]>1)]
    return len(edge_candidates)==0

def is_possible_combination_addition(t, u, v, cs_current, minimal):
    assert len(t.edges_add)<=2 # sanity check
    if len(set(chain(*t.edges_add)))==3 and len(t.edges_remove)==1: # we have a triangle, we add two edges that overlaps at some cn
        # if we remove only 1 edge, that means that we want to need to remove something, and we need to remove 2 edges
        # but maybe we are allowed to remove only ONE edge, so we need to check
        assert len(list(set(chain(*t.edges_add)).difference({u,v})))==1
        third_node = list(set(chain(*t.edges_add)).difference({u,v}))[0] # laaame
        if cs_current[third_node]==1:
            if len([nbr for nbr in minimal.neighbors(third_node) if cs_current[nbr]>1])!=1:
                return False    
        
    # second impossible case is rectangle, e.g. we add (t1,u), (t2,v), and we can remove (t1,t2).
    # If cs(t1)==cs(t2)==1 we 
    # could remove an extra edge, which is bad.
    if len(set(chain(*t.edges_add))) == 4:
        t1, t2 = list(set(chain(*t.edges_add)).difference({u,v}))
        if minimal.has_edge(t1,t2) and cs_current[t1]==1 and cs_current[t2]==1:            
            if frozenset([t1, t2]) in t.edges_remove and len(t.edges_remove)==2:
                return False
    return True
            
def is_possible_combination_removal(t,u,v,cs_current):
    for cn in set(chain(*t.edges_remove)):
        if cn==u or cn==v:
            continue
        if frozenset([u,cn]) in t.edges_remove and frozenset([v,cn]) in t.edges_remove and cs_current[cn]==2:
            return False
    return True
        

In [11]:
def transformations_filter(t, k, cs, cs_all, graph):
    for e1 in t.edges_add:
        e = list(e1)
        if k[e[0]]<k[e[1]]:
            cs[e[0]]+=1
        if k[e[0]]>k[e[1]]:
            cs[e[1]]+=1
        if k[e[0]]==k[e[1]]:
            cs[e[0]]+=1
            cs[e[1]]+=1
    for e1 in t.edges_remove:
        e = list(e1)
        if k[e[0]]<k[e[1]]:
            cs[e[0]]-=1
        if k[e[0]]>k[e[1]]:
            cs[e[1]]-=1
        if k[e[0]]==k[e[1]]:
            cs[e[0]]-=1
            cs[e[1]]-=1
    for n in cs.keys():
        if cs[n]<1:
            return False
        
    edge_candidates_added = len([(u,v) for u,v in t.edges_add if 
                           ((k[u]==k[v] and cs.get(u, cs_all[u])>1 and cs.get(v, cs_all[v])>1)
                           or (k[u]>k[v] and cs.get(v, cs_all[v])>1) or (k[u]<k[v] and cs.get(u, cs_all[u])>1))
                      and frozenset([u,v]) not in t.edges_remove])
    if edge_candidates_added!=0:
        return False
    
    nb_nodes_set = set(chain(*[graph.neighbors(x) for x in set(chain(*t.edges_add))]))
    
    for u,v in graph.subgraph(set(cs.keys()).union(nb_nodes_set)).edges():
        if (((k[u]==k[v] and cs.get(u, cs_all[u])>1 and cs.get(v, cs_all[v])>1)
                           or (k[u]>k[v] and cs.get(v, cs_all[v])>1) or (k[u]<k[v] and cs.get(u, cs_all[u])>1))
                      and frozenset([u,v]) not in t.edges_remove):
            return False

    return True


In [12]:
def fix_transformations(tr, k, cs, minimal, graph):
    edges_add = [tuple(x) for x in tr.edges_add]
    edges_remove = [tuple(x) for x in tr.edges_remove]
    final_set = set()
#     for e1 in edges_add:
#         for e2 in edges_add:
#             if e1!=e2:
#                 u1,v1 = e1
#                 u2,v2 = e2
#                 comb = [(u1,u2),(u1,v2), (v1,u2), (v1,v2)]
#                 for c in comb:
#                     if len(set(c))==1:
#                         continue
#                     if not c in edges_remove and minimal.has_edge(*c):    
#                         cs_r = recompute_cs(edges_add, edges_remove, k, cs)
#                         c_1,c_2 = c
#                         if ((k[c_1]==k[c_2] and cs_r[c_1]>1 and cs_r[c_2]>1)
#                            or (k[c_1]>k[c_2] and cs_r[c_2]>1)
#                             or (k[c_2]>k[c_1] and cs_r[c_1]>1)):
#                             edges_remove.append((c_1,c_2)) 
                            
#     cs_r = recompute_cs(edges_add, edges_remove, k, cs)
#     for e1 in edges_remove:
#         for e2 in edges_remove:
#             if e1!=e2:
#                 u1,v1 = e1
#                 u2,v2 = e2
#                 comb = [(u1,u2),(u1,v2), (v1,u2), (v1,v2)]
#                 for c1 in comb:
#                     if len(set(c1))==1:
#                         c = list(c1)[0]
#                         assert cs_r[c]>=0 #sanity check, again
#                         if cs_r[c]==0:
#                             for nbr in graph.neighbors(c):
#                                 if minimal.has_edge(nbr,c):
#                                     continue
#                                 if k[nbr]>=kcores[c]:
#                                         t = GraphTransformation(edges_add+[(nbr,c)], edges_remove,tr.type)
#                                         final_set.add(t)        
    final_set.add(GraphTransformation(edges_add, edges_remove,tr.type)) # hopefully, will filter out on the next step
    return final_set

def transformations_product_compatible(l1,l2, k, cs, minimal, graph): 
    transformations_set = set()
    for t1 in l1:
        for t2 in l2:
            if t1==t2: continue
            if (not t1.legal) and (not t2.legal):
                continue
            new_tr = [transforms_join(t1, t2)]
            
#             new_tr = fix_transformations(tr, k, cs, minimal, graph)    
            for t in new_tr:
                nodes = set()
                for e1 in set(t.edges_add):
                    nodes.update(e1)
                for e1 in set(t.edges_remove):
                    nodes.update(e1)
                if transformations_filter(t, k, {node:cs[node] for node in nodes},cs, minimal):
                    transformations_set.add(t)
                    
#                 if transformations_filter(t, k, {node:cs[node] for node in nodes},cs, minimal):
#                     transformations_set.add(t)
    return transformations_set

In [13]:
# G.edges()

In [14]:
# G.edge_subgraph([(0,1),(0,3),(0,2)]).edges()

In [15]:
def inverse_transform(tr):
    if "add" in tr.type:
        return GraphTransformation(tr.edges_remove, tr.edges_add, "remove")
    return GraphTransformation(tr.edges_remove, tr.edges_add, "add")


In [16]:
from random import choice, shuffle

def get_greedy_minimal_choice(original_graph:nx.Graph):
    graph = original_graph.copy()
#     graph_nodes = list(graph.nodes())
#     shuffle(graph_nodes)
#     mapping = dict()
#     for i,n in enumerate(graph_nodes):
#         mapping[n] = i
#     graph = nx.relabel_nodes(graph, mapping)
    kcores = nx.core_number(graph)
    cs = get_core_strengths(graph, kcores)
    while not test_minimal(graph, kcores):
        edge_candidates = [(u,v) for u,v in graph.edges() if 
                           (kcores[u]==kcores[v] and cs[u]>1 and cs[v]>1)
                           or (kcores[u]>kcores[v] and cs[v]>1) or (kcores[u]<kcores[v] and cs[u]>1)]
        edge_rem = choice(edge_candidates) #[0]
        graph.remove_edge(*edge_rem)
        cs = get_core_strengths(graph, kcores)
    return graph



In [17]:
import numpy as np
from tqdm.notebook import trange
from itertools import product, chain
from random import choice
import time


def sample_uniformly_minimal(minimal:nx.Graph, graph:nx.Graph, dmax, iterations = 1000, progress_bar = True):
    kcores = nx.core_number(graph)
    assert test_minimal(minimal, kcores), "First argument should be skeletal core!"
    cs = get_core_strengths(graph, kcores)
    skip = False
    transformations_history = []
    skipped_counter = 0
    states_history = []
    
    transformations_set_k = dict()
    for k in set(kcores.values()):
        transformations_set_k[k] = set()
    last_change = -1
    if progress_bar:
        pbar = tqdm(range(iterations), leave=False)
    else:
        pbar = range(iterations)
    for i in pbar:
        cs_current = get_core_strengths(minimal, kcores)
        if not skip:
            if last_change==-1:
                for k in set(kcores.values()):
                    transformations_set_k[k] = set()
            else:
                transformations_set_k[last_change] = set()

#             transformations_set = set()
            for e in graph.edges(): # consider additions. Add 1 edge from the original graph, try to remove necessary edges
                if minimal.has_edge(*e):
                    continue
                u, v  = e
                if last_change!=-1 and last_change!=min(kcores[u],kcores[v]):
                    continue

                if cs_current[v] == 1:
                    u, v = v, u # for convenience.
                if cs_current[u]>1 and cs_current[v]>1:
                    continue # cannot remove any edge from u or v, hence cannot add e

                if kcores[u]>kcores[v]:
                    if cs_current[u]==1 and cs_current[v]>1:
                        # this edge is useless
                        continue
                    if cs_current[v]==1 and cs_current[u]==1:
                        # cs_current[u] == 1 also, because we switched. But we don't really care
                        # now we need to find if we can remove some edges from v to make up for the addition of (u,v)
                        for nbr in minimal.neighbors(v):
                            if nbr==u: continue
                            if (kcores[nbr]>kcores[v]) or (cs_current[nbr]>1 and kcores[nbr]==kcores[v]):
                                t = GraphTransformation([e],[(v, nbr)],'add')
                                transformations_set_k[kcores[v]].add(t)
                            if cs_current[nbr] == 1 and kcores[nbr]==kcores[v]: # go deeper, 1-level cascade
                                for nbr2 in graph.neighbors(nbr):
                                    if minimal.has_edge(nbr, nbr2) or nbr2==v:
                                        continue
                                    if (kcores[nbr2]==kcores[nbr] and cs_current[nbr2]>1) or (kcores[nbr2]>kcores[nbr]):
                                        t = GraphTransformation([e,(nbr,nbr2)],[(v, nbr)],'add')
                                        transformations_set_k[kcores[v]].add(t)

                elif kcores[v]>kcores[u]:
                    if cs_current[v]==1 and cs_current[u]>1:
                        # this edge is useless
                        continue

                    if cs_current[v]==1:
                        # cs_current[u] == 1 also, because we switched. But we don't really care
                        # now we need to find if we can remove some edges from u to make up for the addition of (u,v)
                        for nbr in minimal.neighbors(u):
                            if (kcores[nbr]>kcores[u]) or (cs_current[nbr]>1 and kcores[nbr]==kcores[u]):
                                t = GraphTransformation([e],[(u, nbr)],'add')
                                transformations_set_k[kcores[u]].add(t)
                                if cs_current[nbr] == 1 and kcores[nbr]==kcores[u]: # go deeper, 1-level cascade
                                    for nbr2 in graph.neighbors(nbr):
                                        if minimal.has_edge(nbr, nbr2) or nbr2==u:
                                            continue
                                        if (kcores[nbr2]==kcores[nbr] and cs_current[nbr2]>1) or (kcores[nbr2]>kcores[nbr]):
                                            t = GraphTransformation([e,(nbr,nbr2)],[(u, nbr)],'add')
                                            transformations_set_k[kcores[u]].add(t)
                else:
                    # k[v]==k[u]            
                    assert kcores[u]==kcores[v] #sanity check2
                    if cs_current[u] == 1 and cs_current[v]>1: 
                        for nbr in minimal.neighbors(u):
                            if cs_current[nbr]>1 and kcores[nbr]>=kcores[u]:
                                t = GraphTransformation([e],[(u, nbr)],'add')
                                transformations_set_k[kcores[u]].add(t)
                            if cs_current[nbr] == 1 and kcores[nbr]==kcores[u]: # go deeper, 1-level cascade
                                    for nbr2 in graph.neighbors(nbr):
                                        if minimal.has_edge(nbr, nbr2) or nbr2==u:
                                            continue
                                        if (kcores[nbr2]==kcores[nbr] and cs_current[nbr2]>1) or (kcores[nbr2]>kcores[nbr]):
                                            t = GraphTransformation([e,(nbr,nbr2)],[(u, nbr)],'add')
                                            transformations_set_k[kcores[u]].add(t)
                    elif cs_current[u] > 1 and cs_current[v] == 1: 
                        for nbr in minimal.neighbors(v):
                            if cs_current[nbr]>1 and kcores[nbr]>=kcores[v]:
                                t = GraphTransformation([e],[(u, nbr)],'add')
                                transformations_set_k[kcores[v]].add(t)
                            if cs_current[nbr] == 1 and kcores[nbr]==kcores[v]: # go deeper, 1-level cascade
                                for nbr2 in graph.neighbors(nbr):
                                    if minimal.has_edge(nbr, nbr2) or nbr2==v:
                                        continue
                                    if (kcores[nbr2]==kcores[nbr] and cs_current[nbr2]>1) or (kcores[nbr2]>kcores[nbr]):
                                        t = GraphTransformation([e,(nbr,nbr2)],[(v, nbr)],'add')
                                        transformations_set_k[kcores[v]].add(t)
                    else:
                        left_side, right_side = set(), set()
                        
                        for nbr in minimal.neighbors(u):
                            if (kcores[nbr]>kcores[v]) or (cs_current[nbr]>1 and kcores[nbr]==kcores[v]):
                                t = GraphTransformation([e],[(u, nbr)],'add')
                                left_side.add(t)
                            if cs_current[nbr] == 1 and kcores[nbr]==kcores[u]: # go deeper, 1-level cascade
                                for nbr2 in graph.neighbors(nbr):
                                    if minimal.has_edge(nbr, nbr2) or nbr2==u:
                                        continue
                                    if (kcores[nbr2]==kcores[nbr] and cs_current[nbr2]>1) or (kcores[nbr2]>kcores[nbr]):
                                        t = GraphTransformation([e,(nbr,nbr2)],[(u, nbr)],'add')
                                        left_side.add(t)
                                    if kcores[nbr2]==kcores[nbr] and cs_current[nbr2]==1:
                                        if all([cs_current[nbr3]==1 for nbr3 in minimal.neighbors(nbr2) if kcores[nbr3]==kcores[nbr2]]):
                                            t = GraphTransformation([e,(nbr,nbr2)],[(u, nbr)],'add')
                                            left_side.add(t)

                                v_neigbours = set([x for x in set(minimal.neighbors(v))
                                       if kcores[x]==kcores[v] and cs_current[x]==1])
                                nbr_allowed2add = [nbr2hop for nbr2hop in set(graph.neighbors(nbr)).difference(set(minimal.neighbors(nbr)))
                                                  if nbr2hop in v_neigbours]

                                for nbr2hop in nbr_allowed2add:
                                        t = GraphTransformation([e, (nbr, nbr2hop)],[(u, nbr)] , 'add') # cannot remove any edge from nbr anyway, so will be minimal
                                        left_side.add(t)
                                            
                        for nbr in minimal.neighbors(v):
                            if nbr==u: continue
                            if (kcores[nbr]>kcores[v]) or (cs_current[nbr]>1 and kcores[nbr]==kcores[v]):
                                t = GraphTransformation([e],[(v, nbr)],'add')
                                right_side.add(t)
                            if cs_current[nbr] == 1 and kcores[nbr]==kcores[v]: # go deeper, 1-level cascade
                                for nbr2 in graph.neighbors(nbr):
                                    if minimal.has_edge(nbr, nbr2) or nbr2==v:
                                        continue
                                    if (kcores[nbr2]==kcores[nbr] and cs_current[nbr2]>1) or (kcores[nbr2]>kcores[nbr]):
                                        t = GraphTransformation([e,(nbr,nbr2)],[(v, nbr)],'add')
                                        right_side.add(t)
                                    if kcores[nbr2]==kcores[nbr] and cs_current[nbr2]==1:
                                            if all([cs_current[nbr3]==1 for nbr3 in minimal.neighbors(nbr2) if kcores[nbr3]==kcores[nbr2]]):
                                                t = GraphTransformation([e,(nbr,nbr2)],[(v, nbr)],'add')
                                                right_side.add(t)
                                                
                                                
                                u_neigbours = set([x for x in set(minimal.neighbors(u))
                                           if kcores[x]==kcores[u] and cs_current[x]==1])
                                nbr_allowed2add = [nbr2hop for nbr2hop in set(graph.neighbors(nbr)).difference(set(minimal.neighbors(nbr)))
                                                  if nbr2hop in u_neigbours]
                                for nbr2hop in nbr_allowed2add:
                                        t = GraphTransformation([e,(nbr, nbr2hop)], [(v, nbr)], 'add') # cannot remove any edge from nbr anyway, so will be minimal
                                        right_side.add(t)
                        transformations_set_k[kcores[v]].update(transformations_product_compatible(left_side, right_side, kcores, cs_current, minimal, graph))
#                         transformations_set.update(transformations_product_compatible(left_side, right_side, kcores, cs_current, minimal, graph))
            for e in minimal.edges(): # consider removal
                
                if (cs_current[e[0]]>1 and cs_current[e[1]]>1): # sanity check!
                    return minimal, transformations_history, states_history, False
                u,v = e
                if last_change!=-1 and last_change!=min(kcores[u],kcores[v]):
                    continue
                if cs_current[v] == 1:
                    u, v = v, u
                if kcores[u]<kcores[v]: # need to fix only for u
    #                 assert cs_current[u] == 1
                        #if cs_current[u] == 1:  # it`s impossible to have other case; but anyway              
                        # cs_current[u] == 1
                        # we need to add some edges to u and account for the cascades
                    for nbr in graph.neighbors(u):
                        if minimal.has_edge(u,nbr) or kcores[nbr]<kcores[u]:
                            continue
                        if (kcores[nbr]==kcores[u] and cs_current[nbr] > 1) or kcores[nbr]>kcores[u]: # cannot remove any edges from nbr! 
                            t = GraphTransformation([(u, nbr)], [e],'remove')
                            transformations_set_k[kcores[u]].add(t)
                        else: # may have cascades!
                            nbr_allowed2remove = [(nbr2hop, nbr) for nbr2hop in minimal.neighbors(nbr) 
                                                  if (cs_current[nbr2hop]>1 and kcores[nbr2hop]==kcores[nbr]) or kcores[nbr2hop]>kcores[nbr]]
                            # so now we can remove one edge from nbr_allowed2remove, also remove e and add 
                            if len(nbr_allowed2remove)!=0:
                                combinations = product(nbr_allowed2remove, [e])
                                for c in combinations:
                                    t = GraphTransformation([(u, nbr)], c, 'remove')
                                    transformations_set_k[kcores[u]].add(t)
                            else:
                                t = GraphTransformation([(u, nbr)], [e], 'remove') # cannot remove any edge from nbr anyway, so will be minimal
                                transformations_set_k[kcores[u]].add(t)
                if kcores[v]<kcores[u]: # need to fix only for v

                    if cs_current[v] == 1:                
                        # cs_current[u] == 1
                        # we need to add some edges to u and account for the cascades
                        for nbr in graph.neighbors(v):
                            if minimal.has_edge(v,nbr) or kcores[nbr]<kcores[v]:
                                continue
                            elif (kcores[nbr]==kcores[v] and cs_current[nbr] > 1) or kcores[nbr]>kcores[v]: # cannot remove any edges from nbr! 
                                t = GraphTransformation([(v, nbr)], [e], 'remove')
                                transformations_set_k[kcores[v]].add(t)
                            else: # may have cascades!
                                nbr_allowed2remove = [(nbr2hop, nbr) for nbr2hop in minimal.neighbors(nbr)
                                                      if (cs_current[nbr2hop]>1 and kcores[nbr2hop]==kcores[nbr]) or kcores[nbr2hop]>kcores[nbr]]
                                # so now we can remove one edge from nbr_allowed2remove, also remove e and add add 
                                if len(nbr_allowed2remove)!=0:
                                    combinations = product(nbr_allowed2remove, [e])
                                    for c in combinations:
                                        t = GraphTransformation([(v, nbr)], c,'remove') # cannot remove any edge from nbr anyway, so will be minimal
                                        transformations_set_k[kcores[v]].add(t)
                                else:
                                    t = GraphTransformation([(v, nbr)], [e], 'remove') # cannot remove any edge from nbr anyway, so will be minimal
                                    transformations_set_k[kcores[v]].add(t)
                if kcores[v]==kcores[u]:
                    transformations_u_set = set()
                    transformations_v_set = set()
                    if cs_current[v] > 1:
                        # cs_current[u] == 1
                        # we need to add some edges to u and account for the cascades
                        for nbr in graph.neighbors(u):
                            if minimal.has_edge(u,nbr) or kcores[nbr]<kcores[u]:
                                continue
                            elif (kcores[nbr]==kcores[u] and cs_current[nbr] > 1) or kcores[nbr]>kcores[u]: # cannot remove any edges from nbr! 
                                t = GraphTransformation([(u, nbr)], [e],'remove_eq')
                                transformations_set_k[kcores[v]].add(t)
                            else: # may have cascades!
                                nbr_allowed2remove = [(nbr2hop, nbr) for nbr2hop in minimal.neighbors(nbr) 
                                                      if ((cs_current[nbr2hop]>1 
                                                          and kcores[nbr2hop]==kcores[nbr]) or (kcores[nbr2hop]>kcores[nbr]))
                                                     and (nbr2hop!=v or (nbr2hop==v and (cs_current[v]>2 or kcores[v]>kcores[nbr])))
                                                     ]
                                # so now we can remove one edge from nbr_allowed2remove, also remove e and add add 
                                if len(nbr_allowed2remove)!=0:
                                    combinations = product(nbr_allowed2remove, [e])
                                    for c in combinations:
                                        t = GraphTransformation([(u,nbr)], c,'remove_eq2') # cannot remove any edge from nbr anyway, so will be minimal
                                        transformations_set_k[kcores[v]].add(t)
                                else:
                                    t = GraphTransformation([(u, nbr)], [e], 'remove_eq3') # cannot remove any edge from nbr anyway, so will be minimal
                                    transformations_set_k[kcores[v]].add(t)
                                    
                                nbr_allowed2remove_u = [(nbr2hop, nbr) for nbr2hop in minimal.neighbors(nbr) 
                                                  if ((cs_current[nbr2hop]>1 
                                                      and kcores[nbr2hop]==kcores[nbr]) or (kcores[nbr2hop]>kcores[nbr]))
                                                 and (nbr2hop==v and cs_current[v]==2)
                                                 ] # might be rewritted better, but should work anyway
                                if len(nbr_allowed2remove_u)!=0:
                                    combinations = list(product(nbr_allowed2remove_u, [e]))
                                    assert len(combinations)==1
                                    comb = combinations[0]
                                    t = GraphTransformation([(u,nbr)], comb,'remove_comb') # cannot remove any edge from nbr anyway, so will be minimal
                                    transformations_u_set.add(t)


                    if cs_current[u]==1 and cs_current[v]==1:
                        for nbr in graph.neighbors(u):
                            if minimal.has_edge(u,nbr) or kcores[nbr]<kcores[u]:
                                continue
                            if (cs_current[nbr] > 1 and kcores[nbr]==kcores[u]) or kcores[nbr]>kcores[u]: # cannot remove any edges from nbr! 
                                t = GraphTransformation([(u, nbr)], [e], 'remove_comb')
                                transformations_u_set.add(t)
                            else: # may have cascades!
                                nbr_allowed2remove = [(nbr2hop, nbr) for nbr2hop in minimal.neighbors(nbr) 
                                                  if (cs_current[nbr2hop]>1 and kcores[nbr2hop]==kcores[nbr]) or (kcores[nbr2hop]>kcores[nbr])]
                                # so now we can remove one edge from nbr_allowed2remove, also remove e and add add 
                                if len(nbr_allowed2remove)!=0:
                                    combinations = product(nbr_allowed2remove, [e])
                                    for c in combinations:
                                        t = GraphTransformation([(u, nbr)],c, 'remove_comb') # cannot remove any edge from nbr anyway, so will be minimal
                                        transformations_u_set.add(t)
#                                 else:
                                t = GraphTransformation([(u, nbr)], [e], 'remove_comb') # cannot remove any edge from nbr anyway, so will be minimal
                                transformations_u_set.add(t)

                                v_neigbours = set([x for x in set(graph.neighbors(v)).difference(minimal.neighbors(v))
                                                       if kcores[x]==kcores[v] and cs_current[x]==1])
                                nbr_allowed2remove_e = [(nbr2hop, nbr) for nbr2hop in minimal.neighbors(nbr)
                                                      if nbr2hop in v_neigbours]
                                if len(nbr_allowed2remove_e)!=0:
                                    combinations = product(nbr_allowed2remove_e, [e])
                                    for c in combinations:
                                        t = GraphTransformation([(v, nbr)],c, 'remove_comb') # cannot remove any edge from nbr anyway, so will be minimal
                                        transformations_u_set.add(t)
                                    
                                    
                    if len(transformations_u_set)!=0:
                        for nbr in graph.neighbors(v):
                            if minimal.has_edge(v,nbr) or kcores[nbr]<kcores[v]:
                                continue
                            if (kcores[nbr]==kcores[v] and cs_current[nbr] > 1) or kcores[nbr]>kcores[v]: # cannot remove any edges from nbr! 
                                t = GraphTransformation([(v, nbr)], [e], 'remove_comb')
                                transformations_v_set.add(t)
                            else: # may have cascades!
                                nbr_allowed2remove = [(nbr2hop, nbr) for nbr2hop in minimal.neighbors(nbr)
                                                  if ((cs_current[nbr2hop]>1 
                                                      and kcores[nbr2hop]==kcores[nbr]) or (kcores[nbr2hop]>kcores[nbr]))]
                                # so now we can remove one edge from nbr_allowed2remove, also remove e and add add 
                                if len(nbr_allowed2remove)!=0:
                                    combinations = product(nbr_allowed2remove, [e])
                                    for c in combinations:
                                        t = GraphTransformation([(v, nbr)],c, 'remove_comb') # cannot remove any edge from nbr anyway, so will be minimal
                                        transformations_v_set.add(t)
#                                 else:
                                t = GraphTransformation([(v, nbr)], [e], 'remove_comb') # cannot remove any edge from nbr anyway, so will be minimal
                                transformations_v_set.add(t)

                                u_neigbours = set([x for x in set(graph.neighbors(u)).difference(minimal.neighbors(u))
                                                       if kcores[x]==kcores[u] and cs_current[x]==1])
                                nbr_allowed2remove_e = [(nbr2hop, nbr) for nbr2hop in minimal.neighbors(nbr)
                                                      if nbr2hop in u_neigbours]
                                if len(nbr_allowed2remove_e)!=0:
                                    combinations = product(nbr_allowed2remove_e, [e])
                                    for c in combinations:
                                        t = GraphTransformation([(v, nbr)],c, 'remove_comb') # cannot remove any edge from nbr anyway, so will be minimal
                                        transformations_v_set.add(t)

                        if len(transformations_v_set)!=0:
                            transformations_set_k[kcores[v]].update(transformations_product_compatible(transformations_u_set, transformations_v_set, kcores, cs_current, minimal, graph))
#                             transformations_set.update(transformations_product_compatible(transformations_u_set, transformations_v_set, kcores, cs_current, minimal, graph))
        transformations_set = set()
        for k, items in transformations_set_k.items():
            transformations_set.update(items)
        assert dmax>len(transformations_set), len(transformations_set)
#         print(len(transformations_set))
        if len(transformations_history)!=0:
            if not inverse_transform(transformations_history[-1][0]) in transformations_set:
                print('aaaaa')
                return minimal, transformations_history, states_history, False
#             assert inverse_transform(transformations_history[-1]) in transformations_set
#             transformations_set.add(inverse_transform(transformations_history[-1][0]))
        p = 1-len(transformations_set)/dmax
#         p = 0
        if progress_bar:
            pbar.set_description("p={:06.4f},|s| = {}".format(p,len(transformations_set)))
        states_history.append(len(transformations_set))
        if np.random.uniform()>p:
            tr = choice(list(transformations_set))
            transformations_history.append((tr,minimal.copy()))
            minimal.add_edges_from([list(e) for e in tr.edges_add])
            minimal.remove_edges_from([list(e) for e in tr.edges_remove])
            if not test_minimal(minimal, kcores):
                print('bbbb')
                return minimal, transformations_history, states_history, False
            skip = False
            last_change = min(min([kcores[n1] for n1 in set(chain(*tr.edges_add))]),
                             min([kcores[n1] for n1 in set(chain(*tr.edges_remove))]))
        else:
            skip = True
            skipped_counter+=1
#         tr = choice(list(transformations_set))
#         minimal.add_edges_from([list(e) for e in tr.edges_add])
#         minimal.remove_edges_from([list(e) for e in tr.edges_remove])
    return minimal, transformations_history, states_history, True 



In [18]:
from tqdm.contrib.concurrent import process_map

In [ ]:
GRAPHS = {'inf-power':"data/inf-power.mtx",
          'protein':"data/protein/bio-yeast-protein-inter.edges",
          'erdos':"data/erdos/ca-Erdos992.mtx",
         'auto_as19990309':"data/as-733/as19990309.txt",
          'auto_as19990111':"data/as-733/as19990111.txt",
          'wiki':"data/wiki/soc-wiki-Vote.mtx",
         'norwegian': 'data/norwegian/net1m_2011-08-01.txt',
         'netscience':'data/dimacs10-netscience/out.dimacs10-netscience',
         'moreno_innovation':"data/moreno_innovation/out.moreno",
#          'inf_power':'data/inf-power/inf-power.mtx',
         'GrQc':'data/ca_GrQc/ca-GrQc.mtx',
         'auto_as19980318':'data/as19980318.txt',
         'auto_as19971108':'data/as19971108.txt',
         'tech_routers':'data/routers/tech-routers-rf.mtx',
         "ca_grq":"data/ca_GrQc/ca-GrQc.mtx",
         'oregon':"data/oregon_331/oregon1_010331.txt",
         'arena':"data/arenas-jazz/out.arenas-jazz",
#          'moreno_oz':"data//moreno_oz/out.moreno_oz_oz",
          'moreno_oz':"data/moreno_oz/out.moreno_oz_ozcsv",
         "hamsterer":"data/hamsterer/soc-hamsterster.edges",
          "openflights":"data/openflights/inf-openflights.edges",
          "tech_pgp":"data/pgp/tech-pgp.edges",
          'dmela':"data/dmela/bio-dmela.mtx",
          "whois":"data/WHOIS/tech-WHOIS.mtx",
          'webspam':"data/web-spam/web-spam.mtx",
          "gnutella08":"data/gnutalla08/p2p-Gnutella08.txt",
#           'notredame':"data/web-NotreDame.txt",
          'email_auall':"data/Email-EuAll.txt",
          'email_enron':"data/Email-Enron.txt",
          'musae_git':"data/git_web_ml/musae_git_edges.csv",
          'musae_facebook':"data/facebook_large/musae_facebook_edges.csv",
#           'fb_pages_artist':"data/fb-pages-artist/fb-pages-artist.edges",
#           'gowalla':"data/soc-gowalla/soc-gowalla.mtx",
#           'slashdot':"data/soc-slashdot/soc-slashdot.mtx",
          'ca-HepPh':'data/ca-HepPh.mtx', 
         }

G = nx.read_edgelist(GRAPHS['musae_facebook'])
G.remove_edges_from(nx.selfloop_edges(G))
G.number_of_edges()

In [32]:
# G = nx.karate_club_graph()
# graph = G.copy()


mapping = dict()
for i, e in enumerate(G.edges()):
    mapping[e] = i
    
def one_hot(subgraph, mapping):
    onehot = [0 for x in range(len(mapping))]
    for e in subgraph.edges():
        onehot[mapping[e]] = 1
    return tuple(onehot)

In [33]:
graph = G.copy()

In [37]:
from tqdm.contrib.concurrent import process_map

In [20]:
# from tqdm.notebook import tqdm
# import pickle

# sampled_graphs = dict()
# states_history_dict = dict()
# proportions = [0.2]#0.05, 0.1]#,0.4,0.5]#,0.05,0.06,0.1]#, 1.2, 1.4, 1.6, 1.8, 2]
# flag2 = True
# for itr in tqdm(reversed(proportions), total = len(proportions)):
#     sampled_graphs[itr] = []
#     states_history_dict[itr] = []
#     for j in trange(5):
#         itr_edges = int(graph.number_of_edges()*itr)
#         skeletal_random = get_greedy_minimal_choice(graph)
#         result, history, states_history,flag = sample_uniformly_minimal(skeletal_random.copy(),
#                                                                         graph.copy(), dmax=800, iterations=itr_edges, progress_bar=True)
#         if not flag:
#             flag2 = False
#             break
#         sampled_graphs[itr].append(result)
#         states_history_dict[itr].append(states_history)
#     if not flag2:
#         break
    
#         sampled_graphs[itr].append(sample_uniformly(graph.copy(), itr_edges).copy())
        
# #     n_sk, history, states_history,flag = sample_uniformly_minimal(skeletal.copy(), graph, dmax=30000, iterations=100)
#     if not flag:
#         break

In [42]:
from tqdm.notebook import tqdm
new_skeletal_cores = []
import pickle

skeletal = get_greedy_minimal_choice(graph.copy())
states_history_list = []
tr_history_list = []

sampled_graphs = dict()
states_history_dict = dict()

# for i in trange(1000000):
#     n_sk, history, states_history , flag = sample_uniformly_minimal(get_greedy_minimal_choice(graph), graph, dmax=75, iterations=200, progress_bar=False)
# proportions = [0.01,0.02,0.03,0.05,0.1,0.2,0.5,0.7,1.0]
proportions = [0.5,1.0,2.0,3.0,5.0,7.0,10.0]
# proportions = [0.01,0.02,0.03]
# proportions = [0.5]
# proportions = [0.2]
# proportions = [0.05]

# norwegian 512
# auto_as19990111 800
graph_name = "karate"
# graph_name = "auto_as19990111" 

for itr in tqdm(reversed(proportions), total = len(proportions)):

    itr_edges = int(graph.number_of_edges()*itr)
    def wrapper(i):
        n_sk, _, _, flag = sample_uniformly_minimal(get_greedy_minimal_choice(graph), graph.copy(), 
                                         dmax=800, iterations=itr_edges, progress_bar=False)
        return (n_sk, [0],[0], flag)

    r = process_map(wrapper, range(200), chunksize=5, max_workers=10)
    sampled_graphs[itr] = []
    states_history_dict[itr] = []
    
    for j, elem in enumerate(r):            
        n_sk, history, states_history,flag = elem
        assert flag==True
        #         sampled_graphs[itr].append(n_sk)
        #         states_history_dict[itr].append(states_history)

#         with open("ricky_ds_samples/{}/{}_{}.history".format(graph_name, itr,j),'wb') as f:
#             pickle.dump(states_history, f)
#         with open("ricky_ds_samples/{}/{}_{}.history".format(graph_name, itr,j),'wb') as f:
#             pickle.dump(n_sk, f)
        with open("paper_samples/{}/{}_{}.history".format(graph_name, itr,j),'wb') as f:
            pickle.dump(n_sk, f)




  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]